In [ ]:
# !pip install OpenDartReader

In [ ]:
# from financial_data_making import financial_dict
import OpenDartReader
import pandas as pd 
import os
import glob

df = pd.read_csv('esg_finance_data.csv', dtype={'corp_code': 'str', 'stock_code': 'str'})
df['year'] = ['2020'] * 847 + ['2021'] * 847 + ['2022'] * 847
df['year'] = df['year'].astype('int')
print(df.dtypes)
print(df.shape, len(df.corp_code.unique())) #847
df.head(10)

In [ ]:
# 매출액 데이터

api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)

# 매출액
# 손익계산서
def sales_docu(code, year):
    sales = dart.finstate(code, year)
    # real_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')].iloc[0, :] # 매출액 
    c_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['thstrm_amount'].values # 매출액 # 문제1
    p_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['frmtrm_amount'].values # t-1기 매출액 # 문제 2

    div_cash = dart.report(code, '배당', year) #(연결)현금배당성향(%)
    c_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['thstrm'].values[0] # 순이익이 마이너스일 경우 0% # 배당금 / 당기순이익
    p_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['frmtrm'].values[0]

    holder = dart.report(code, '최대주주', year) # 보통주 계 + 우선수 계 (%)
    ratio_stakeholder = holder[(holder['stock_knd']=='보통주')&(holder['nm']=='계')].values # 문제3

    exec = dart.report(code, '임원', year) #'사외이사' / 전체 임원 수   
    out_exec = exec[exec['ofcps'] == '사외이사'].shape[0]
    exec = exec.shape[0]

    employee = dart.report(code, '직원', year)['rgllbr_co'].to_list() # 총 직원 수
    
    output = {'sales': c_sales, 
              'pre_sales': p_sales, 
              'cash_div': c_div, 
              'pre_cash_civ': p_div,
              'stakeholder' : ratio_stakeholder,
              'num_outexecutives': out_exec,
              'num_executives': exec,
              'num_employee': employee}
    
    return output

In [ ]:
# 데이터 수집 (1)

data = []
good_index = []
bad_index = []

for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
    try:
        data.append(sales_docu(code, year))
        good_index.append(index)
    except:
        bad_index.append(index)
print(len(good_index), len(bad_index)) # 

# data downloading
with open ('s_docu.txt', 'w', encoding='utf-8') as f:
    for x in df.iloc[good_index, 0].values:
        f.write(f'{x}\n')
    f.close()
    
with open ('f_docu.txt', 'w', encoding='utf-8') as f:
    for x in df.iloc[bad_index, 0].values:
        f.write(f'{x}\n')
    f.close()

In [ ]:
# 연결 재무제표 데이터

api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)

def report_docu(code, year):
    lst = dart.list(code, start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기
    
    # 변경1: 원하는 년도의 사업 보고서 고르기 
    report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(year)))].iloc[0, 5]
    url = dart.sub_docs(report_num) #사업보고서 번호를 통해 url 요청
    html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소
    
    docu = pd.read_html(html) # 데이터 읽어오기 
    if docu[1].empty | docu[3].empty: # [1] 재무상태표 [3] 손익계산서
        print('Some document cannot be extracted')
    else:
        fin_state = docu[1].rename(columns={'Unnamed: 0': 'tag'}) # 재무상태표

        asset = fin_state[fin_state['tag'] == '자산총계'] # 자산 총계 DF
        debt= fin_state[fin_state['tag'] == '부채총계'] # 부채 총계 DF
        intangible = fin_state[fin_state['tag'].str.contains('무형자산')] # 무형자산 DF

        income = docu[3].rename(columns={'Unnamed: 0': 'tag'}) # 손익계산서DF

        profit = income[income['tag'].str.contains('영업이익')] # 영업이익(손실) 
        profit_tax = income[income['tag'].str.contains('법인세비용차감전')] # t기 법인세비용차감전순이익(손실)
        
    return pd.concat([asset, debt, intangible, profit, profit_tax], axis=0).reset_index(drop=True)                                                                                           

In [3]:
# 데이터 수집 (2-1)
# year를 2022으로 고정해서 2022, 2021, 2020 데이터 수집

good_index, bad_index = [], []

# 847개 회사 unique한 stock_code를 이용 
# O(file 1) = 0.5 secs
# 약 425초, 7분 예상

for index, code in enumerate(df['stock_code'].unique()): 
    try:
        report_docu(code, 2022).to_csv(f'./dataset_2022/{code}_{str(2022)}.csv', encoding='utf8') # dataset 파일 안에 code에 따라 저장
        print(f'{index}, Good!') # check if it works well
        good_index.append(index)
    except:
        print(f'{index}, bad...') # check if there is a problem
        bad_index.append(index)
        
print(len(good_index), len(bad_index)) 

10, Good!
11, Good!
12, Good!
13, Good!
14, Good!
15, Good!
16, Good!
17, Good!
18, bad...
19, Good!
20, Good!
21, Good!
22, Good!
23, Good!
24, Good!
25, Good!
26, Good!
27, Good!
28, Good!
29, Good!
30, Good!
31, bad...
32, Good!
33, Good!
34, Good!
35, Good!
36, Good!
37, Good!
38, Good!
39, Good!
40, Good!
41, Good!
42, Good!
43, bad...
44, Good!
45, Good!
46, Good!
47, Good!
48, Good!
49, Good!
50, Good!
51, Good!
52, Good!
53, Good!
54, Good!
55, Good!
56, Good!
57, bad...
58, Good!
59, Good!
60, Good!
61, Good!
62, Good!
63, Good!
64, Good!
65, Good!
66, Good!
67, Good!
68, Good!
69, Good!
70, Good!
71, Good!
72, Good!
73, Good!
74, Good!
75, Good!
76, Good!
77, Good!
78, Good!
79, Good!
80, Good!
81, Good!
82, Good!
83, Good!
84, bad...
85, Good!
86, Good!
87, Good!
88, Good!
89, Good!
90, Good!
91, Good!
92, Good!
93, bad...
94, Good!
95, Good!
96, Good!
97, Good!
98, Good!
99, Good!
100, Good!
101, Good!
102, bad...
103, Good!
104, Good!
105, bad...
106, Good!
107, Good!
108,

In [4]:
# 데이터 수집 (2-2)
# 사업보고서 2020를 통해서 2019년도 자료 구하기 가능

good_index, bad_index = [], []

for index, code in enumerate(df['stock_code'].unique()):
    try:
        print(f'{index}, Good!')
        good_index.append(index)
        report_docu(code, 2020).to_csv(f'./dataset_2020/{code}_{str(2020)}.csv', encoding='utf8') # dataset 파일 안에 code에 따라 저장
    except:
        print(f'{index}, bad...')
        bad_index.append(index)
        
print(len(good_index), len(bad_index)) 

0, Good!
1, Good!
2, Good!
3, Good!
4, Good!
5, Good!
6, Good!
7, Good!
7, bad...
8, Good!
9, Good!
10, Good!
11, Good!
12, Good!
13, Good!
14, Good!
15, Good!
16, Good!
17, Good!
18, Good!
18, bad...
19, Good!
20, Good!
21, Good!
22, Good!
23, Good!
23, bad...
24, Good!
25, Good!
26, Good!
27, Good!
28, Good!
29, Good!
30, Good!
31, Good!
31, bad...
32, Good!
33, Good!
34, Good!
35, Good!
36, Good!
37, Good!
38, Good!
39, Good!
40, Good!
41, Good!
42, Good!
43, Good!
44, Good!
45, Good!
46, Good!
47, Good!
48, Good!
49, Good!
50, Good!
51, Good!
52, Good!
52, bad...
53, Good!
54, Good!
55, Good!
56, Good!
57, Good!
58, Good!
59, Good!
60, Good!
61, Good!
61, bad...
62, Good!
63, Good!
64, Good!
65, Good!
66, Good!
67, Good!
68, Good!
69, Good!
70, Good!
71, Good!
71, bad...
72, Good!
72, bad...
73, Good!
74, Good!
75, Good!
76, Good!
77, Good!
78, Good!
79, Good!
80, Good!
81, Good!
82, Good!
83, Good!
84, Good!
84, bad...
85, Good!
86, Good!
87, Good!
88, Good!
88, bad...
89, Good!
8

In [5]:
# 누락 데이터 확인 # dataset_2022 # 185개 기업 데이터 누락

file_path1 = './dataset_2022'

codes = []
for f in glob.glob(os.path.join(file_path1, '*.csv')):
    code = f.split('_')[1][-6:]
    codes.append(code)
    
missed_code = [code for code in list(df['stock_code'].unique()) if code not in codes]

print(len(codes))
print(len(missed_code)) # 185개 기업 데이터 누락

df1 = pd.DataFrame(missed_code, columns=['stock_code'])
df2 = pd.merge(df1, df[['stock_code', 'corp_name']], on='stock_code', how='left')
df2 = df2.drop_duplicates(['stock_code'])

print(df2.head(3))

662
185
  stock_code corp_name
0     000990     DB하이텍
3     003490      대한항공
6     005380     현대자동차


In [6]:
# 데이터 추가수집 (2-1*) # dataset_2022 #누락된 연결재무제표 데이터 보충

api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)

def missed_report_docu(code, year):
    lst = dart.list(code, start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기

    # 변경1: 원하는 년도의 사업 보고서 고르기 
    report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(year)))].iloc[0, 5]
    url = dart.sub_docs(report_num) #사업보고서 번호를 통해 url 요청
    html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소

    docu = pd.read_html(html) # 데이터 읽어오기 
    if docu[1].empty | docu[3].empty: # [1] 재무상태표 [3] 손익계산서
        print('Some document cannot be extracted')
    else:
        for index, content in enumerate(docu):
            if content.iloc[0,0] =='연결 재무상태표':
                fin_state = docu[index+1].rename(columns={'Unnamed: 0': 'tag'}) # 재무상태표

                asset = fin_state[fin_state['tag'] == '자산총계'] # 자산 총계 DF
                debt= fin_state[fin_state['tag'] == '부채총계'] # 부채 총계 DF
                intangible = fin_state[fin_state['tag'].str.contains('무형자산')] # 무형자산 DF
                
            elif content.iloc[0,0] =='연결 손익계산서':
                income = docu[index+1].rename(columns={'Unnamed: 0': 'tag'}) # 손익계산서DF
                
                profit = income[income['tag'].str.contains('영업이익')] # 영업이익(손실) 
                profit_tax = income[income['tag'].str.contains('법인세비용차감전')] # t기 법인세비용차감전순이익(손실)
        
    return pd.concat([asset, debt, intangible, profit, profit_tax], axis=0).reset_index(drop=True)             

good_index, bad_index = [], []

for index, code in enumerate(df2['stock_code'].unique()): # 185개 기업
    try:
        missed_report_docu(code, 2022).to_csv(f'./dataset_2022/{code}_{str(2022)}.csv', encoding='utf8') # dataset 파일 안에 code에 따라 저장        
        print(f'{index}, Good!') # check if it works well
        good_index.append(index)
    except:
        print(f'{index}, bad...') # check if there is a problem
        bad_index.append(index)
        
print(len(good_index), len(bad_index))

0, Good!
1, bad...
2, Good!
3, bad...
4, bad...
5, bad...
6, bad...
7, bad...
8, bad...
9, Good!
10, bad...
11, bad...
12, bad...
13, Good!
14, bad...
15, bad...
16, bad...
17, bad...
18, bad...
19, bad...
20, bad...
21, bad...
22, bad...
23, bad...
24, bad...
25, bad...
26, bad...
27, bad...
28, Good!
29, bad...
30, bad...
31, bad...
32, bad...
33, bad...
34, bad...
35, bad...
36, bad...
37, bad...
38, bad...
39, bad...
40, bad...
41, bad...
42, bad...
43, bad...
44, bad...
45, bad...
46, bad...
47, bad...
48, bad...
49, Good!
50, bad...
51, bad...
52, bad...
53, bad...
54, bad...
55, bad...
56, bad...
57, bad...
58, bad...
59, bad...
60, bad...
61, bad...
62, bad...
63, bad...
64, bad...
65, bad...
66, bad...
67, bad...
68, bad...
69, Good!
70, bad...
71, bad...
72, bad...
73, bad...
74, bad...
75, bad...
76, bad...
77, bad...
78, Good!
79, bad...
80, bad...
81, bad...
82, bad...
83, bad...
84, bad...
85, bad...
86, bad...
87, bad...
88, bad...
89, bad...
90, bad...
91, bad...
92, ba

In [7]:
# 누락 데이터 확인 # dataset_2020

file_path2 = './dataset_2020'
codes = []
for f in glob.glob(os.path.join(file_path2, '*.csv')):
    code = f.split('_')[1][-6:]
    codes.append(code)
    
missed_code = [code for code in list(df['stock_code'].unique()) if code not in codes]

print(len(codes))
print(len(missed_code)) # 213개 기업 데이터 누락

df3 = pd.DataFrame(missed_code, columns=['stock_code'])
df4 = pd.merge(df3, df[['stock_code', 'corp_name']], on='stock_code', how='left')
df4 = df4.drop_duplicates(['stock_code'])
df4.head(3)

634
213


,stock_code,corp_name
0,096770,SK이노베이션
3,000990,DB하이텍
6,001740,SK네트웍스


In [8]:
# 데이터 추가수집 (2-2*) # dataset_2020 #누락된 연결재무제표 데이터 보충
good_index, bad_index = [], []

for index, code in enumerate(df4['stock_code'].unique()): # 213개 기업
    try:
        missed_report_docu(code, 2020).to_csv(f'./dataset_2022/{code}_{str(2020)}.csv', encoding='utf8') # dataset 파일 안에 code에 따라 저장
        print(f'{index}, Good!') # check if it works well
        good_index.append(index)
    except:
        print(f'{index}, bad...') # check if there is a problem
        bad_index.append(index)
        
print(len(good_index), len(bad_index))

0, bad...
1, Good!
2, bad...
3, bad...
4, bad...
5, bad...
6, Good!
7, bad...
8, bad...
9, bad...
10, bad...
11, bad...
12, bad...
13, bad...
14, bad...
15, bad...
16, bad...
17, Good!
18, bad...
19, bad...
20, Good!
21, bad...
22, bad...
23, Good!
24, bad...
25, bad...
26, bad...
27, bad...
28, bad...
29, Good!
30, bad...
31, bad...
32, bad...
33, bad...
34, bad...
35, bad...
36, bad...
37, bad...
38, bad...
39, bad...
40, bad...
41, bad...
42, bad...
43, bad...
44, bad...
45, bad...
46, bad...
47, bad...
48, bad...
49, bad...
50, bad...
51, bad...
52, bad...
53, bad...
54, bad...
55, bad...
56, bad...
57, Good!
58, bad...
59, bad...
60, bad...
61, bad...
62, bad...
63, bad...
64, bad...
65, bad...
66, bad...
67, bad...
68, Good!
69, bad...
70, bad...
71, bad...
72, bad...
73, bad...
74, bad...
75, bad...
76, bad...
77, bad...
78, bad...
79, bad...
80, bad...
81, bad...
82, bad...
83, bad...
84, bad...
85, bad...
86, bad...
87, bad...
88, bad...
89, bad...
90, Good!
91, bad...
92, bad

In [ ]:
lst = dart.list('003490', start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기

# 변경1: 원하는 년도의 사업 보고서 고르기 
report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(2020)))].iloc[0, 5]
url = dart.sub_docs(report_num) #사업보고서 번호를 통해 url 요청
html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소

docu = pd.read_html(html) # 데이터 읽어오기 
if docu[1].empty | docu[3].empty: # [1] 재무상태표 [3] 손익계산서
    print('Some document cannot be extracted')
else:
    # output = docu[2] # 2 or 4 # 1 or 4 
    for index, content in enumerate(docu):
        if content.iloc[0,0] =='연결 재무상태표':
            fin_state = docu[index+1].rename(columns={'Unnamed: 0': 'tag'}) # 재무상태표

            asset = fin_state[fin_state['tag'] == '자산총계'] # 자산 총계 DF
            debt= fin_state[fin_state['tag'] == '부채총계'] # 부채 총계 DF
            intangible = fin_state[fin_state['tag'].str.contains('무형자산')] # 무형자산 DF
            
        elif content.iloc[0,0] =='연결 손익계산서':
            income = docu[index+1].rename(columns={'Unnamed: 0': 'tag'}) # 손익계산서DF
            
            profit = income[income['tag'].str.contains('영업이익')] # 영업이익(손실) 
            profit_tax = income[income['tag'].str.contains('법인세비용차감전')] # t기 법인세비용차감전순이익(손실)
       
    
pd.concat([asset, debt, intangible, profit, profit_tax], axis=0).reset_index(drop=True)     
# docu

# docu = pd.read_html(url.iloc[0,1])
# docu

# docu = pd.read_html(html)
# docu